In [ ]:
from mayavi import mlab
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from PIL import Image, ImageDraw
# from tqdm import tqdm, trange
# import time
import warnings
from scipy.ndimage.filters import gaussian_filter

savePath = "/Users/nguyendat/Documents/GitHub/polar_vortice/polar_vortice/notebooks/Animation/PLANET1/"
### IMPORTANT: create offscreen rendering to save computation
mlab.options.offscreen = True
    
########## Construct Mayavi spherical mesh #######################
### Create a sphere
r = 1
pi = np.pi
cos = np.cos
sin = np.sin

### Construct the spherical mesh grid
phi0 = np.linspace(0, np.pi, 200)
theta0 = np.linspace(0, 2*np.pi, 200)
phi, theta = np.meshgrid(phi0, theta0)

x = r * sin(phi) * cos(theta)
y = r * sin(phi) * sin(theta)
z = r * cos(phi)

xsize, ysize = x.shape[0], x.shape[1]

#################### FUNCTIONS FOR CREATING PLANETARY-SCALE WAVES ##########
############# convert latitude to pixel location ####################
def lat(coord,n=ysize):
    return abs(coord-90)/180*n # pixel location
############## Create Planetary-scale waves ##############################
### unit: hour, rotational period: 5 hour
def plWave(x, value, t, w=xsize, f=0, amplitude=0.4, base=0.5, RP=5): # planetary scale waves
    # create the sine-wave modulation, such that the sine wave looks the same after
    # every rotational period RP
    # w is the spatial period equals to the pixel size
    sine = (amplitude/2)*np.sin(2*np.pi/w * (x + (t/RP)*w) + f*np.pi/180) + base
    # return the original value + sine wave
    return value + sine

In [ ]:
###############################
## Save each frame
save = False
# save = True
###############################
# draw_animation = False
draw_animation = True
#### save animation as gif
save_animation = False
# save_animation = True
###############################
modelname = 'BD_B'
###############################
%matplotlib inline
warnings.filterwarnings("ignore")

### Initialization
Fband= 1.0
Fambient = 0.55
# Fpolar = 0.35

### periods
Pband = 3.0
Ppol = 10.0

# inclination
# inclin = 0
inclin = +10
# inclin = +45
# inclin = +80
# inclin = -10

# scaled amplitudes
a1 = 1
a2 = 1
a3 = 1
a4 = 1
apol = 1

###### Create an atmosphere mesh 
### Edit the property of bands here!
def atmos_mesh(x,t=0):
    im = Fambient*np.ones(x.shape)
    # lat = theta0*180/pi-90    
    for xx in range(xsize):
        for yy in range(ysize):
            #### Band-features
            ### The lat() function converts latitude to pixel coordinate
            ### The order is reverse: input the latitudes from large to small using
            ### the format: lat(LARGE) < yy < lat(SMALL)
            # equatorial bands 
            if lat(45) < yy < lat(30): im[xx,yy] = a2*plWave(xx,Fband, f=10, t=t, base=0, RP=Pband/2)
            if lat(20) < yy < lat(5): im[xx,yy] = a1*plWave(xx, Fband, f=60, t=t, base=0, RP=Pband)
            if lat(89) < yy < lat(65): im[xx,yy] = apol*plWave(xx, Fband, f=-30, t=t, base=0, RP=Ppol)
            # if lat(-15) < yy < lat(-25): im[xx,yy] = a3*plWave(xx, Fband, f=45, t=t, base=Fambient)
            # if lat(-40) < yy < lat(-50): im[xx,yy] = a4*plWave(xx, Fband, w=xsize/2, f=70, t=t, base=Fambient)
            # # polar bands
            # if lat(80) < yy < lat(60): im[xx,yy] = plWave(xx, Fpolar, f=40, t=ti, RP=15)
            # if lat(-60) < yy < lat(-80): im[xx,yy] = plWave(xx, Fpolar, f=0, t=ti, RP=15)
    return im

####################### CREATE ANIMATION ######################################
### Create list of images
imarray = []
fluxarray = []
frame_no = 100
gray_array = []
t0, t1 = 0,15
cmap = 'plasma'

### create time_array in hours unit
time_array = np.linspace(t0,t1,frame_no)
### Create matplotlib figure
plt.close('all')
fig2, ax2 = plt.subplots()
### create 3d figure
mfig = mlab.figure(1, bgcolor=(0, 0, 0), fgcolor=(0, 0, 0), size=(500, 500))
    
### Start looping through the timesteps array
for i in range(frame_no):
    ti = time_array[i]
    ######### Create 3d mesh in Mayavi THEN paste into matplotlib figures ##########

    ### configuring the 3d view and camera
    mlab.view(azimuth=180.0, elevation=90.0-inclin, distance=2.0, roll=90)

    ### Do the meshplot 
    s = atmos_mesh(x,t=ti)
    sphere = mlab.mesh(x, y, z, scalars=s, colormap=cmap, vmin=0, vmax=1)

    ### Set up cosmetics
    sphere.actor.property.lighting = False
    mfig.scene.parallel_projection = True
    mfig.scene.reset_zoom()
    camera = mfig.scene.camera
    camera.zoom(1.57)

    ### Take screenshot as array object
    imgmap = mlab.screenshot(figure=mfig, mode='rgba', antialiased=True)
    mlab.clf()

    ######### Pasting into matplotlib figures ############################
    ### Show mayavi im as Matplotlib object and draw animation
    if draw_animation: 
        imobj = ax2.imshow(imgmap, vmin=0, vmax=1, animated=True, cmap=cmap)
        if i == 0:
            ax2.imshow(imgmap, vmin=0, vmax=1, cmap=cmap) # show an initial one first
            # plt.show()
    else:
        imobj = ax2.imshow(imgmap, vmin=0, vmax=1, cmap=cmap)
    imarray.append([imobj])

    if save:
        plt.margins(0, 0), plt.draw(), plt.axis('off')
        fig.savefig(savePath+'ani_%04d.jpeg'%i, format='jpeg', bbox_inches = 'tight', pad_inches=0)
    
    ### convert map into grayscale
    imgray = imgmap[:,:,0] * 0.2989 + imgmap[:,:,1] * 0.587 + imgmap[:,:,2] * 0.114
    gray_array.append([imgray])
    if ((i+1)/frame_no*100)%10 == 0.0: 
        print('%i%% done'%((i+1)/frame_no*100))

mlab.close()

### plotting
if draw_animation:
    ani = animation.ArtistAnimation(fig2, imarray)
    if save_animation:
        ### To save the animation using Pillow as a gif
        writer = animation.PillowWriter(fps=60,
                                        metadata=dict(artist='Me'),
                                        bitrate=1800)
        ani.save(savePath+'%s_i=%i.gif'%(modelname, inclin), writer=writer)
    # plt.show()
    plt.close()

In [ ]:
# save = True
save = False

%matplotlib inline
# time = [1, 25, 50, 75]
time = [1, 10]
plt.close('all')
font = 8

for t in time:
    sumfluxmean = 0
    frameim = gray_array[t][0]
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(9,3), dpi=120)
    axs[0].imshow(frameim, cmap='gray', vmin=0, vmax=1)
    fsum = np.sum(frameim)
    sumfluxmean+=fsum
    axs[0].set_title('Original,F=%.5e'%fsum, fontsize=font)

    sig = 25
    blur = gaussian_filter(frameim, sigma=sig)
    fsum = np.sum(blur)
    sumfluxmean+=fsum
    axs[1].imshow(blur, cmap='gray', vmin=0, vmax=1)
    axs[1].set_title(r'Gaussian-filter $\sigma$=%i, F=%.5e'%(sig, fsum), fontsize=font)

    sig = 50
    blur = gaussian_filter(frameim, sigma=sig)
    fsum = np.sum(blur)
    sumfluxmean+=fsum
    axs[2].imshow(blur, cmap='gray', vmin=0, vmax=1)
    axs[2].set_title(r'Gaussian-filter, $\sigma$=%i, F=%.5e'%(sig, fsum), fontsize=font)
    
    fig.suptitle('Frame %i, F=%.5e'%(t, sumfluxmean/4))
    if save:
        fig.savefig(savePath+'%s_[GaussObs_frame%i]_i=%i.png'%(modelname, t, inclin), format='png', dpi=300)

In [ ]:
### Find flux by applying gaussian filter sigma=50
flux = []
for frame in range(frame_no):
    image = gaussian_filter(gray_array[frame][0], sigma=50)
    flux.append(np.sum(image))

In [ ]:
# save = True
save = False 

flux = np.array(flux)
plt.close('all'), plt.figure(dpi=120)
normflux = flux - (flux.max()+flux.min())/2
normflux/=2*normflux.max()
normflux+=1
plt.plot(time_array, normflux)
plt.title(r'i=%i$^\circ$, p=%.1f, %.1f, %.1f hr'%(inclin,Pband/2, Pband, Ppol))
plt.xlabel('Time (hours)'), plt.ylabel('Normalized Flux')
if save:
    plt.savefig(savePath+'%s_[flux]_i=%i.png'%(modelname, inclin), format='png', dpi=300)